
## **Minecraft Block Palette Generator:**



In [207]:
from google.colab import drive
from pathlib import Path

# 1. Mount your Google Drive
drive.mount('/content/drive')

# 2. Define the path to your folder (change 'MyProject' to your folder name)
# Note: 'My Drive' usually has a space in it
folder_path = Path("/content/drive/MyDrive/llm_week2/llm_week2_project2")
file_path = folder_path / 'palettes_selenium.csv'
import pandas as pd

# 1. Load the CSV
df = pd.read_csv(file_path)

# 2. Group by 'Palette_ID' and collect 'Block_Name' into SETS
# We use .apply(set) here
grouped_data = df.groupby('Palette_ID')['Block_Name'].apply(set)

# 3. Convert to a list of sets
palette_sets = grouped_data.tolist()

# 4. Print the result
print(f"Found {len(palette_sets)} unique palettes.")
print("-" * 30)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Found 891 unique palettes.
------------------------------


In [208]:
import torch
import torch.nn.functional as F
import random
def matton(a):
  return a[0]*27 +a[1]
from numpy import shape
from math import exp

In [209]:
blocks = sorted(list(set().union(*palette_sets)))
stoi = {s:i for i,s in enumerate(blocks)}
itos = {i:s for s,i in stoi.items()}

In [210]:
print(stoi)
len(stoi)

{'Acacia Leaves': 0, 'Acacia Log': 1, 'Acacia Planks': 2, 'Amethyst Block': 3, 'Ancient Debris': 4, 'Andesite': 5, 'Azalea Leaves': 6, 'Bamboo Block': 7, 'Bamboo Mosaic': 8, 'Barrel': 9, 'Basalt': 10, 'Beacon': 11, 'Bedrock': 12, 'Bee Nest': 13, 'Beehive': 14, 'Beehive Top': 15, 'Birch Leaves': 16, 'Birch Log': 17, 'Birch Planks': 18, 'Black Concrete': 19, 'Black Concrete Powder': 20, 'Black Glazed Terracotta': 21, 'Black Shulker Box': 22, 'Black Stained Glass': 23, 'Black Terracotta': 24, 'Black Wool': 25, 'Blackstone': 26, 'Blast Furnace': 27, 'Blue Concrete': 28, 'Blue Concrete Powder': 29, 'Blue Glazed Terracotta': 30, 'Blue Ice': 31, 'Blue Shulker Box': 32, 'Blue Stained Glass': 33, 'Blue Terracotta': 34, 'Blue Wool': 35, 'Bone Block': 36, 'Bookshelf': 37, 'Brain Coral Block': 38, 'Bricks': 39, 'Brown Concrete': 40, 'Brown Concrete Powder': 41, 'Brown Glazed Terracotta': 42, 'Brown Mushroom Block': 43, 'Brown Shulker Box': 44, 'Brown Stained Glass': 45, 'Brown Terracotta': 46, 'Br

342

In [211]:
random.seed(200000)
random.shuffle(palette_sets)
train = palette_sets[:711]
dev = palette_sets[712:800]
test = palette_sets[801:]

In [212]:
N = torch.zeros((len(stoi), len(stoi)), dtype=torch.int32)
for w in train:
  for e1 in w:
    for e2 in w:
      if e2!=e1:
        i1 = stoi[e1]
        i2 = stoi[e2]
        N[i1,i2] += 1

In [213]:
N

tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 5,  ..., 1, 1, 1],
        [0, 5, 0,  ..., 0, 0, 0],
        ...,
        [0, 1, 0,  ..., 0, 0, 1],
        [0, 1, 0,  ..., 0, 0, 5],
        [0, 1, 0,  ..., 1, 5, 0]], dtype=torch.int32)

In [214]:
P = (N).float()+0.001
P /= P.sum(1, keepdims=True)

In [215]:
g = torch.Generator().manual_seed(2147483647)

for i in range(3):

  out = []
  ix = random.randint(0,len(stoi))
  out.append(itos[ix])
  while True:
    p = P[ix]
    i1 = torch.multinomial(p, num_samples=6, replacement=False, generator=g)# i am sampling 6 so that if it samples itself other 5 can be considered
    for ele in i1:
      if itos[ele.item()] not in out and len(out)<6:
        out.append(itos[ele.item()])
    break
  print(' , '.join(out))

Spawner , Copper Grate , Tube Coral Block , Dead Horn Coral Block , Cracked Polished Blackstone Bricks , Oxidized Cut Copper
Cobbled Deepslate , Oxidized Cut Copper , Black Concrete Powder , Cobblestone , Stone Bricks , Pale Oak Planks
Stripped Mangrove Log , Spruce Planks , Mangrove Planks , Stripped Dark Oak Log , Nether Bricks , Red Wool


In [216]:
log_likelihood = 0.0
n = 0

for w in train:
  for e1 in w:
    for e2 in w:
      if e2!=e1:
        ix1 = stoi[e1]
        ix2 = stoi[e2]
        prob = P[ix1, ix2]
        logprob = torch.log(prob)
        log_likelihood += logprob
        n += 1
print(f'{log_likelihood=}')
nll = -log_likelihood
print(f'{nll=}')
print(f'{nll/n}')

log_likelihood=tensor(-83714.4922)
nll=tensor(83714.4922)
3.9395055770874023


In [217]:
g = torch.Generator().manual_seed(2147483647)
W = torch.randn((len(stoi), len(stoi)), generator=g, requires_grad=True)

In [218]:
for k in range(6):
  loss1=0
  for w in train:
    xs, ys = [], []
    for e1 in w:
      for e2 in w:
        if e2!=e1:
          ix1 = stoi[e1]
          ix2 = stoi[e2]
          xs.append(ix1)
          ys.append(ix2)

    xs = torch.tensor(xs)
    ys = torch.tensor(ys)
    num = xs.nelement()
    # forward pass
    xenc = F.one_hot(xs, num_classes=len(stoi)).float()
    logits = xenc @ W
    counts = logits.exp()
    probs = counts / counts.sum(1, keepdims=True)
    logprob = probs[torch.arange(num), ys].log()
    loss1 += -logprob.mean() #+ 0.01*(W**2).mean()
  loss1 = loss1/len(train)
  print(loss1.item())
  # backward pass
  W.grad = None
  loss1.backward()

  # update
  W.data += -600 * W.grad

6.328130722045898
6.2355852127075195
6.149304389953613
6.0684404373168945
5.9926629066467285
5.921748161315918


In [219]:
g = torch.Generator().manual_seed(2147483647)

for i in range(3):

  out = []
  ix = random.randint(0,len(stoi))
  out.append(itos[ix])
  while True:
    xenc = F.one_hot(torch.tensor([ix]), num_classes=len(stoi)).float()
    logits = xenc @ W
    counts = logits.exp()
    p = counts / counts.sum(1, keepdims=True)
    i1 = torch.multinomial(p, num_samples=6, replacement=False, generator=g)
    for row in i1:
      for ele in row:
        if itos[ele.item()] not in out and len(out)<6:
          out.append(itos[ele.item()])
    break
  print(' , '.join(out))

Stripped Birch Log , Copper Grate , Cracked Polished Blackstone Bricks , Oxidized Cut Copper , Dead Horn Coral Block , Nether Gold Ore
Soul Soil , Pink Glazed Terracotta , Copper Block , Black Concrete Powder , Honeycomb Block , Oxidized Cut Copper
Bamboo Mosaic , Crafting Table , Copper Block , Spruce Planks , Orange Concrete Powder , Green Concrete


In [220]:
#loss without smoothing
loss1=0
for w in dev:
  xs, ys = [], []
  for e1 in w:
    for e2 in w:
      if e2!=e1:
        ix1 = stoi[e1]
        ix2 = stoi[e2]
        xs.append(ix1)
        ys.append(ix2)

  xs = torch.tensor(xs)
  ys = torch.tensor(ys)
  num = xs.nelement()
  # forward pass
  xenc = F.one_hot(xs, num_classes=len(stoi)).float()
  logits = xenc @ W
  counts = logits.exp()
  probs = counts / counts.sum(1, keepdims=True)
  logprob = probs[torch.arange(num), ys].log()
  loss1 += -logprob.mean() #+ 0.01*(W**2).mean()
loss1 = loss1/len(dev)
print(loss1.item())

5.9563398361206055


In [221]:
#loss in probs
log_likelihood = 0.0
n = 0

for w in dev:
  for e1 in w:
    for e2 in w:
      if e2!=e1:
        ix1 = stoi[e1]
        ix2 = stoi[e2]
        prob = P[ix1, ix2]
        logprob = torch.log(prob)
        log_likelihood += logprob
        n += 1
print(f'{log_likelihood=}')
nll = -log_likelihood
print(f'{nll=}')
print(f'{nll/n}')

log_likelihood=tensor(-18348.1152)
nll=tensor(18348.1152)
6.950043678283691


In [222]:
g = torch.Generator().manual_seed(2147483647)
W1 = torch.randn((len(stoi), len(stoi)), generator=g, requires_grad=True)

In [223]:
for k in range(5):
  loss1=0
  for w in train:
    xs, ys = [], []
    for e1 in w:
      for e2 in w:
        if e2!=e1:
          ix1 = stoi[e1]
          ix2 = stoi[e2]
          xs.append(ix1)
          ys.append(ix2)

    xs = torch.tensor(xs)
    ys = torch.tensor(ys)
    num = xs.nelement()
    # forward pass
    xenc = F.one_hot(xs, num_classes=len(stoi)).float()
    logits = xenc @ W1
    counts = logits.exp()
    probs = counts / counts.sum(1, keepdims=True)
    logprob = probs[torch.arange(num), ys].log()
    loss1 += -logprob.mean() + 0.00001*(W1**2).mean()#
  loss1 = loss1/len(train)
  print(loss1.item())
  # backward pass
  W.grad = None
  loss1.backward()

  # update
  W1.data += -100 * W1.grad
  #training it further from a point reduces loss for train but increases loss for dev (overfitting?)
  #I have further saved that point and loaded it as T

6.328140735626221
6.312475204467773
6.2815093994140625
6.236001968383789
6.177035808563232


In [224]:
#loss after smoothing
loss1=0
for w in dev:
  xs, ys = [], []
  for e1 in w:
    for e2 in w:
      if e2!=e1:
        ix1 = stoi[e1]
        ix2 = stoi[e2]
        xs.append(ix1)
        ys.append(ix2)

  xs = torch.tensor(xs)
  ys = torch.tensor(ys)
  num = xs.nelement()
  # forward pass
  xenc = F.one_hot(xs, num_classes=len(stoi)).float() # input to the network: one-hot encoding
  logits = xenc @ W1 # predict log-counts
  counts = logits.exp() # counts, equivalent to N
  probs = counts / counts.sum(1, keepdims=True) # probabilities for next character
  logprob = probs[torch.arange(num), ys].log()
  loss1 += -logprob.mean() #+ 0.01*(W**2).mean()
loss1 = loss1/len(dev)
print(loss1.item())

6.131756782531738


In [225]:
weight = folder_path / 'weight.pt'
#torch.save(W1, weight)

In [226]:
T= torch.load(weight) # Final neural net weight

In [230]:
g = torch.Generator().manual_seed(2147483647)

out = []
b = input('Enter a Block: ')
if b in set().union(*palette_sets):
  ix = stoi[b]
else:
  print('We do not have that but check out this block palette')
  ix = random.randint(0,len(stoi))
out.append(itos[ix])
while True:
  xenc = F.one_hot(torch.tensor([ix]), num_classes=len(stoi)).float()
  logits = xenc @ T
  counts = logits.exp()
  p = counts / counts.sum(1, keepdims=True)
  i1 = torch.multinomial(p, num_samples=6, replacement=False)
  for row in i1:
    for ele in row:
      if itos[ele.item()] not in out and len(out)<6:
        out.append(itos[ele.item()])
  break
print(' ')
print(' , '.join(out))

Enter a Block: Spruce Log
 
Spruce Log , Spruce Planks , Stripped Spruce Log , Cobblestone , Deepslate Bricks , Tuff Bricks
